# **Download and Import Dataset**

In [48]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv('2025_09_06_MousedB.csv', sep=';')

print("Columns in df:", df.columns.tolist())

Columns in df: ['Brand', 'Model', 'Size', 'Length (mm)', 'Width (mm)', 'Height (mm)', 'Weight (grams)', 'Shape', 'Hand compatibility', 'Hump placement', 'Front flare', 'Side curvature', 'Thumb rest', 'Ring finger rest', 'Sensor brand', 'Sensor', 'Sensor type', 'Sensor position', 'DPI', 'Polling rate (Hz)', 'Tracking speed (IPS)', 'Acceleration (G)', 'Side buttons', 'Middle buttons', 'Hot-swap switches', 'Switches brand', 'Switches', 'Scroll wheel encoder brand', 'Scroll wheel encoder', 'Material', 'Connectivity']


# **Selection and Clean Data**

In [49]:
features = [
    'DPI',
    'Polling rate (Hz)',
    'Weight (grams)',
    'Length (mm)',
    'Width (mm)',
    'Height (mm)',
    'Side buttons'
]

# ตรวจสอบว่าคอลัมน์เหล่านี้มีอยู่ใน df หรือไม่
existing_features = [f for f in features if f in df.columns]
missing_features = [f for f in features if f not in df.columns]

print("Columns found:", existing_features)
print("Columns missing:", missing_features)

# สร้าง DataFrame เฉพาะคุณสมบัติที่มีอยู่จริง
df_features = df[existing_features].copy()

# ✅ 1a. ตรวจสอบข้อมูลที่หายไปและ % ของข้อมูลที่ใช้ได้/ใช้ไม่ได้
valid_counts = df_features.notna().sum()
total_counts = len(df_features)
percent_valid = (valid_counts / total_counts * 100).round(2)
percent_missing = 100 - percent_valid

print("จำนวนข้อมูลที่ใช้ได้ (%):\n", percent_valid)
print("จำนวนข้อมูลที่หายไป (%):\n", percent_missing)



# ใช้ existing_features แทน features
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_features), columns=existing_features)

# สเกลข้อมูล
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=existing_features)

print("\nข้อมูลหลังจากการสเกล:")
df_scaled.head()

Columns found: ['DPI', 'Polling rate (Hz)', 'Weight (grams)', 'Length (mm)', 'Width (mm)', 'Height (mm)', 'Side buttons']
Columns missing: []
จำนวนข้อมูลที่ใช้ได้ (%):
 DPI                   99.92
Polling rate (Hz)     99.83
Weight (grams)        99.75
Length (mm)          100.00
Width (mm)           100.00
Height (mm)          100.00
Side buttons          99.92
dtype: float64
จำนวนข้อมูลที่หายไป (%):
 DPI                  0.08
Polling rate (Hz)    0.17
Weight (grams)       0.25
Length (mm)          0.00
Width (mm)           0.00
Height (mm)          0.00
Side buttons         0.08
dtype: float64

ข้อมูลหลังจากการสเกล:


,DPI,Polling rate (Hz),Weight (grams),Length (mm),Width (mm),Height (mm),Side buttons
0,0.482632,-0.757271,-0.141154,-0.202779,-0.595220,-0.119875,-0.112955
1,-0.894385,-0.757271,-0.261663,-0.202779,-0.595220,-0.119875,-0.112955
2,0.482632,-0.757271,-0.301833,-0.425520,-0.441788,-0.259140,-0.112955
3,-0.894385,-0.757271,-0.301833,-0.425520,-0.441788,-0.259140,-0.112955
4,-0.894385,-0.757271,-0.100984,-0.425520,-0.441788,-0.259140,-0.112955


# **Dimension Reduction using PCA**

In [50]:
# 1️⃣ เพิ่มมิติข้อมูล (Feature Engineering)
# สร้าง features ใหม่จากข้อมูลเดิมเพื่อให้ PCA มีความหมายมากขึ้น
df_features['Volume'] = df_features['Length (mm)'] * df_features['Width (mm)'] * df_features['Height (mm)']
df_features['Weight_to_Volume'] = df_features['Weight (grams)'] / df_features['Volume']

# ตรวจสอบว่ามีค่า infinite หรือไม่ (จากการหาร)
df_features.replace([np.inf, -np.inf], np.nan, inplace=True)

# 2️⃣ แปลงข้อมูลเชิงหมวดหมู่ (Categorical) เป็นเชิงตัวเลข (One-hot encoding)
categorical_cols = ['Shape', 'Grip type', 'Sensor type']
existing_cats = [col for col in categorical_cols if col in df.columns]

if existing_cats:
    df_cats = pd.get_dummies(df[existing_cats], prefix=existing_cats)
    df_features = pd.concat([df_features, df_cats], axis=1)

# 3️⃣ จัดการ Missing Values
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_features), columns=df_features.columns)

# 4️⃣ สเกลข้อมูลก่อน PCA
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=df_imputed.columns)


# สร้างโมเดล PCA โดยระบุว่าเราต้องการลดมิติเหลือ 2 มิติ
pca = PCA(n_components=2)

# ทำการ fit และ transform ข้อมูล
principal_components = pca.fit_transform(df_scaled)

# สร้าง DataFrame ใหม่สำหรับผลลัพธ์จาก PCA
df_pca = pd.DataFrame(
    data=principal_components,
    columns=['PC1', 'PC2']
)

# เพิ่มข้อมูลชื่อเมาส์และยี่ห้อกลับเข้าไปเพื่อใช้วิเคราะห์
df_pca['Model'] = df['Model']
df_pca['Brand'] = df['Brand']

print("\nผลลัพธ์จากการทำ PCA 5 แถวแรก:")
df_pca.head()






ผลลัพธ์จากการทำ PCA 5 แถวแรก:


,PC1,PC2,Model,Brand
0,-0.936122,-0.265593,T520 Max,AJAZZ
1,-0.768192,-0.682492,T520,AJAZZ
2,-1.099152,-0.255619,G3 V2 Pro,MCHOSE
3,-0.857178,-0.757565,G3 V2,MCHOSE
4,-0.732831,-0.901596,G3 A,MCHOSE


In [51]:
# สร้างกราฟ Scatter Plot แบบ Interactive ด้วย Plotly
fig = px.scatter(
    df_pca,
    x='PC1',
    y='PC2',
    color='Brand', # แยกสีตามยี่ห้อ
    hover_name='Model', # เมื่อเอาเมาส์ไปชี้จะแสดงชื่อรุ่น
    title='ผลการวิเคราะห์ PCA ของข้อมูลเมาส์',
    labels={'PC1': 'แกนคุณลักษณะหลักที่ 1 (PC1)', 'PC2': 'แกนคุณลักษณะหลักที่ 2 (PC2)'}
)

fig.show()

#  **Find Similar Mouse using Cosine Similarity**

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. กำหนด "คลังเมาส์" ของคุณ
my_mouse_collection = ['Viper Mini Signature Edition', 'MAD R Major', 'Susanto-X', 'Y2 Pro']
df_my_collection = df_pca[df_pca['Model'].isin(my_mouse_collection)]

# 2. คำนวณ "โปรไฟล์เมาส์ในอุดมคติ" (จุดศูนย์กลางของคลัง)
ideal_mouse_profile = df_my_collection[['PC1', 'PC2']].mean().values.reshape(1, -1)

print(f"คลังเมาส์ของฉัน:\n{df_my_collection}\n")

# 3. คำนวณ Cosine Similarity
cosine_sim = cosine_similarity(df_pca[['PC1', 'PC2']], ideal_mouse_profile).flatten()
df_pca['Cosine_Similarity_to_Ideal'] = cosine_sim

# 4. จัดอันดับเมาส์ที่ใกล้เคียงที่สุด (ไม่รวมเมาส์ที่มีอยู่แล้วในคลัง)
recommendations = df_pca[~df_pca['Model'].isin(my_mouse_collection)] \
    .sort_values('Cosine_Similarity_to_Ideal', ascending=False) \
    .head(5)

print(f"จุดศูนย์กลาง (Ideal Profile) ในแกน PCA: {ideal_mouse_profile.flatten()}")
print("\n--- 5 อันดับเมาส์แนะนำที่ใกล้เคียงกับสไตล์ของคุณที่สุด ---")
print(recommendations[['Model', 'Brand', 'Cosine_Similarity_to_Ideal']])

# 5. พล็อตกราฟแสดง Ideal Profile
fig.add_scatter(
    x=[ideal_mouse_profile[0, 0]],
    y=[ideal_mouse_profile[0, 1]],
    mode='markers',
    marker=dict(symbol='star', color='red', size=15),
    name='Ideal Profile (จุดศูนย์กลาง)'
)
fig.show()


คลังเมาส์ของฉัน:
          PC1       PC2                         Model   Brand
6   -1.812402  0.887065                     Susanto-X  Pulsar
152 -1.496731  1.485352                        Y2 Pro     VGN
501 -2.056864  1.259486                   MAD R Major     VXE
983 -1.778022  0.876592  Viper Mini Signature Edition   Razer

จุดศูนย์กลาง (Ideal Profile) ในแกน PCA: [-1.78600473  1.12712374]

--- 5 อันดับเมาส์แนะนำที่ใกล้เคียงกับสไตล์ของคุณที่สุด ---
                      Model   Brand  Cosine_Similarity_to_Ideal
149  Miao AM Infinity Mouse   Angry                    1.000000
221                 A9 Plus     ATK                    1.000000
401              JI E-YOOSO     HUO                    0.999999
504                Shark R2  Attack                    0.999985
135             ZEN 8K Mini     PMM                    0.999977


#  **Find Similar Mouse using Euclidean Distance Similarity**

In [53]:
import numpy as np

# 1. กำหนด "คลังเมาส์" ของคุณ
my_mouse_collection = ['Viper Mini Signature Edition', 'MAD R Major', 'Susanto-X', 'Y2 Pro']
df_my_collection = df_pca[df_pca['Model'].isin(my_mouse_collection)]

# 2. คำนวณ "โปรไฟล์เมาส์ในอุดมคติ"
ideal_mouse_profile = df_my_collection[['PC1', 'PC2']].mean().values

# 3. คำนวณ Euclidean Distance ของทุกเมาส์จากจุดศูนย์กลาง
distances = np.linalg.norm(df_pca[['PC1', 'PC2']].values - ideal_mouse_profile, axis=1)
df_pca['Distance_to_Ideal'] = distances

# 4. จัดอันดับเมาส์ที่ใกล้เคียงที่สุด (ไม่รวมเมาส์ที่มีอยู่แล้วในคลัง)
recommendations = df_pca[~df_pca['Model'].isin(my_mouse_collection)] \
    .sort_values('Distance_to_Ideal', ascending=True).head(5)

print(f"จุดศูนย์กลาง (Ideal Profile) ในแกน PCA: {ideal_mouse_profile}")
print("\n--- 5 อันดับเมาส์แนะนำที่ใกล้เคียงกับสไตล์ของคุณที่สุด ---")
print(recommendations[['Model', 'Brand', 'Distance_to_Ideal']])

# 5. พล็อตกราฟแสดง Ideal Profile
fig.add_scatter(
    x=[ideal_mouse_profile[0]],
    y=[ideal_mouse_profile[1]],
    mode='markers',
    marker=dict(symbol='star', color='red', size=15),
    name='Ideal Profile (จุดศูนย์กลาง)'
)
fig.show()


จุดศูนย์กลาง (Ideal Profile) ในแกน PCA: [-1.78600473  1.12712374]

--- 5 อันดับเมาส์แนะนำที่ใกล้เคียงกับสไตล์ของคุณที่สุด ---
           Model     Brand  Distance_to_Ideal
472         M996  Redragon           0.124088
643   F1 Pro Max       ATK           0.128044
124    VK M1 Pro  Valkyrie           0.136857
342       AX5 V2    MCHOSE           0.142778
163  Black Lotus     UNIUS           0.147502
